# California Housing Market : Feature engineering and feature selection
In the previous exercise, we concluded it was worth including more variables in a model. But is this set of variables **the best** we could have chosen ? In this exercises, we'll go further by applying two canonical methods:
* Feature engineering consists in creating more variables from the original dataset
* Feature selection allows to select the best set of features among all the available variables

## The dataset
1. Load the California Housing dataset again and remove the outliers:

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import datasets
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
#pio.renderers.default = "svg" # to be replaced by "iframe" if working on JULIE

In [2]:
# Download data into a pandas DataFrame
data = datasets.fetch_california_housing(data_home=None, download_if_missing=True, return_X_y=False)
dataset = pd.DataFrame(columns=data["feature_names"], data=data["data"])
dataset.loc[:,'Price'] = data["target"]
dataset.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [3]:
# Remove outliers
mask = (dataset['AveRooms'] < 10) & (dataset['AveBedrms'] < 10) & (dataset['Population'] < 15000) & \
    (dataset['AveOccup'] < 10) & (dataset['Price'] < 5)
dataset = dataset.loc[mask,:]

In [4]:
# Basic stats
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 19398

Display of dataset: 


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422



Basics statistics: 


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
count,19398.000000,19398.000000,19398.000000,19398.000000,19398.000000,19398.000000,19398.000000,19398.000000,19398.000000
mean,3.674497,28.496907,5.210648,1.066038,1442.172080,2.944640,35.637872,-119.567484,1.924128
std,1.563397,12.477953,1.168098,0.128846,1077.498768,0.766194,2.142960,2.004793,0.971784
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.750000,32.540000,-124.350000,0.149990
25%,2.525900,18.000000,4.407329,1.005413,805.000000,2.450413,33.930000,-121.770000,1.167000
50%,3.447800,29.000000,5.170038,1.047619,1185.500000,2.842105,34.260000,-118.490000,1.741000
75%,4.583175,37.000000,5.944617,1.096884,1752.000000,3.308127,37.720000,-118.000000,2.485000
max,15.000100,52.000000,9.979167,3.411111,13251.000000,9.954545,41.950000,-114.550000,4.991000



Percentage of missing values: 


MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
Price         0.0
dtype: float64

2. Separate the target from the features

In [5]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Price"

X = dataset.drop(target_variable, axis = 1)
Y = dataset.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
X.head()

Separating labels from features...
...Done.

Y : 
0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: Price, dtype: float64

X :


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


## From linear to non-linear regression
An easy way of implementing a non-linear regression is to create by hand more columns containing non-linear functions of the features.

3. For each explanatory variable, create 3 new columns in $X$ containing the following functions:
* $\textrm{X}^2$
* $\textrm{X}^3$
* $\textrm{X}^4$
* $\frac{1}{\textrm{X}}$
* $\frac{1}{\textrm{X}^2}$

In [6]:
features_list = X.columns
for c in features_list:
    X.loc[:, c + '_2'] = X[c]**2
    X.loc[:, c + '_3'] = X[c]**3
    X.loc[:, c + '_4'] = X[c]**3
    X.loc[:, c + '_inverse'] = 1/X[c]
    X.loc[:, c + '_inverse2'] = 1/(X[c]**2)
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedInc_2,MedInc_3,...,Latitude_2,Latitude_3,Latitude_4,Latitude_inverse,Latitude_inverse2,Longitude_2,Longitude_3,Longitude_4,Longitude_inverse,Longitude_inverse2
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,69.308955,577.010912,...,1434.8944,54353.799872,54353.799872,0.026399,0.000697,14940.1729,-1.826137e+06,-1.826137e+06,-0.008181,0.000067
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,68.913242,572.076387,...,1433.3796,54267.751656,54267.751656,0.026413,0.000698,14937.7284,-1.825689e+06,-1.825689e+06,-0.008182,0.000067
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,52.669855,382.246204,...,1432.6225,54224.761625,54224.761625,0.026420,0.000698,14942.6176,-1.826586e+06,-1.826586e+06,-0.008181,0.000067
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,31.844578,179.702136,...,1432.6225,54224.761625,54224.761625,0.026420,0.000698,14945.0625,-1.827034e+06,-1.827034e+06,-0.008180,0.000067
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,14.793254,56.897815,...,1432.6225,54224.761625,54224.761625,0.026420,0.000698,14945.0625,-1.827034e+06,-1.827034e+06,-0.008180,0.000067


4. Split your dataset into train (80%) and test (20%)

In [7]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



5. Apply the same preprocessing as in the previous exercise

In [8]:
# Preprocessing
print("Preprocessing X_train...")
print(X_train.head())
print()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
print("...Done!")
print(X_train[0:5,:]) # X_train is now a numpy array

Preprocessing X_train...
       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
3235   2.3889       6.0  6.316614   1.294671       992.0  3.109718     36.09   
13981  3.4912       7.0  8.355308   1.554795      2933.0  2.511130     34.85   
9219   1.9464      36.0  4.975510   1.053061       639.0  2.608163     37.12   
10851  3.1667      22.0  3.803838   1.000000      1952.0  2.081023     33.66   
8888   4.2520      31.0  3.978296   1.039389      1985.0  1.595659     34.03   

       Longitude   MedInc_2   MedInc_3  ...  Latitude_2    Latitude_3  \
3235     -119.57   5.706843  13.633078  ...   1302.4881  47006.795529   
13981    -117.46  12.188477  42.552412  ...   1214.5225  42326.109125   
9219     -120.27   3.788473   7.373884  ...   1377.8944  51147.440128   
10851    -117.90  10.027989  31.755632  ...   1132.9956  38136.631896   
8888     -118.49  18.079504  76.874051  ...   1158.0409  39408.131827   

         Latitude_4  Latitude_inverse  Latitude_inverse

In [9]:
print("Preprocessing X_test...")
print(X_test.head())
print()
X_test = scaler.transform(X_test) # don't fit again !
print("...Done!")
print(X_test[0:5,:]) # X_train is now a numpy array

Preprocessing X_test...
       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
17333  5.2990      12.0  7.214932   1.047511      1200.0  2.714932     34.91   
1012   2.6667      44.0  4.541284   1.027523       277.0  2.541284     37.68   
5124   1.5521      30.0  3.850679   1.002262      1966.0  4.447964     33.99   
1845   6.3538      49.0  6.293886   1.017751      1148.0  2.264300     37.90   
4035   3.2154      20.0  4.133444   1.060181      7450.0  1.772122     34.17   

       Longitude   MedInc_2    MedInc_3  ...  Latitude_2    Latitude_3  \
17333    -120.44  28.079401  148.792746  ...   1218.7081  42545.099771   
1012     -121.77   7.111289   18.963674  ...   1419.7824  53497.400832   
5124     -118.26   2.409014    3.739031  ...   1155.3201  39269.330199   
1845     -122.28  40.370774  256.507827  ...   1436.4100  54439.939000   
4035     -118.52  10.338797   33.243368  ...   1167.5889  39896.512713   

         Latitude_4  Latitude_inverse  Latitude_in

6. Train a model including all these features. Do you get better performances than before?

In [10]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [11]:
# Print R^2 scores
print("R2 score on training set : ", regressor.score(X_train, Y_train))
print("R2 score on test set : ", regressor.score(X_test, Y_test))

R2 score on training set :  0.6806483963682184
R2 score on test set :  0.6832246870490242


## Forward selection
This feature engineering trick improved the model's score significantly ! But now, the model is a lot more complex as it uses 32 input features. Do we really need all these features? Let's implement the forward selection method described in this morning's lecture. 

Fortunately, the latest versions of sklearn provide a class that implements forward selection, such that we don't need to code the algorithm by hand 🥳

7. Have a look at the documentation of [SequentialFeatureSelector](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html) and try to understand the following lines of code:

In [12]:
from sklearn.feature_selection import  SequentialFeatureSelector
feature_selector =  SequentialFeatureSelector(regressor, n_features_to_select = 20)
feature_selector.fit(X_train, Y_train)
features_list = X.columns
best_features = features_list[feature_selector.support_]
print("According to the forward selection algorithm, the following features should be kept: ")
print(best_features.to_list())

According to the forward selection algorithm, the following features should be kept: 
['MedInc', 'HouseAge', 'Population', 'Latitude', 'MedInc_inverse2', 'HouseAge_inverse', 'AveRooms_3', 'AveRooms_inverse', 'AveRooms_inverse2', 'AveBedrms_2', 'AveBedrms_inverse', 'Population_inverse2', 'AveOccup_3', 'AveOccup_inverse', 'AveOccup_inverse2', 'Latitude_2', 'Latitude_3', 'Latitude_inverse', 'Latitude_inverse2', 'Longitude_inverse2']


8. Create a DataFrame X_best containing only the best set of features, train a model only with these features and evaluate the performances

In [13]:
X_best = X.loc[:, best_features]

# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X_best, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

# Preprocessing
print("Preprocessing X_train...")
print(X_train.head())
print()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
print("...Done!")
print(X_train[0:5,:]) # X_train is now a numpy array

print("Preprocessing X_test...")
print(X_test.head())
print()
X_test = scaler.transform(X_test) # don't fit again !
print("...Done!")
print(X_test[0:5,:]) # X_train is now a numpy array

# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

# Print R^2 scores
print("R2 score on training set : ", regressor.score(X_train, Y_train))
print("R2 score on test set : ", regressor.score(X_test, Y_test))

Dividing into train and test sets...
...Done.

Preprocessing X_train...
       MedInc  HouseAge  Population  Latitude  MedInc_inverse2  \
3235   2.3889       6.0       992.0     36.09         0.175228   
13981  3.4912       7.0      2933.0     34.85         0.082045   
9219   1.9464      36.0       639.0     37.12         0.263959   
10851  3.1667      22.0      1952.0     33.66         0.099721   
8888   4.2520      31.0      1985.0     34.03         0.055311   

       HouseAge_inverse  AveRooms_3  AveRooms_inverse  AveRooms_inverse2  \
3235           0.166667  252.030501          0.158313           0.025063   
13981          0.142857  583.293888          0.119684           0.014324   
9219           0.027778  123.172247          0.200984           0.040395   
10851          0.045455   55.038428          0.262892           0.069112   
8888           0.032258   62.963842          0.251364           0.063184   

       AveBedrms_2  AveBedrms_inverse  Population_inverse2  AveOccup_3  \


**Thanks to forward selection, we restrained ourselves to only 20 features but still got the same performances as with the 40 initial features !** 🤓

## Advanced feature engineering
Let's make even more advanced feature engineering. Until now, we've included the latitude and longitude as such into the models. However, usually the GPS coordinates are not used rawly, instead we deduce some geographical information from these. Let's use an API that will allows to retrieve the name of the city from the latitude and longitude.

💡 As the calls to the API may be time-consuming, we'll work on a sample of the dataset.

9. Take a sample of your dataset X (the one that contains all the features and not only the best set, because we need the values of Latitude and Longitude). Keep only 150 rows.

In [14]:
X_sample = X.sample(150)
X_sample.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedInc_2,MedInc_3,...,Latitude_2,Latitude_3,Latitude_4,Latitude_inverse,Latitude_inverse2,Longitude_2,Longitude_3,Longitude_4,Longitude_inverse,Longitude_inverse2
1523,5.0499,32.0,6.213873,0.982659,486.0,2.809249,37.90,-122.08,25.501490,128.779974,...,1436.4100,54439.939000,54439.939000,0.026385,0.000696,14903.5264,-1.819423e+06,-1.819423e+06,-0.008191,0.000067
2704,3.5282,31.0,5.260563,1.017606,959.0,3.376761,32.83,-115.57,12.448195,43.919722,...,1077.8089,35384.466187,35384.466187,0.030460,0.000928,13356.4249,-1.543602e+06,-1.543602e+06,-0.008653,0.000075
2047,6.8162,15.0,7.383621,1.060345,766.0,3.301724,36.72,-119.72,46.460582,316.684622,...,1348.3584,49511.720448,49511.720448,0.027233,0.000742,14332.8784,-1.715932e+06,-1.715932e+06,-0.008353,0.000070
11174,2.9891,26.0,4.651394,1.003984,1121.0,2.233068,33.82,-117.97,8.934719,26.706768,...,1143.7924,38683.058968,38683.058968,0.029568,0.000874,13916.9209,-1.641779e+06,-1.641779e+06,-0.008477,0.000072
1763,3.6719,45.0,4.177778,0.888889,702.0,2.600000,37.95,-122.34,13.482850,49.507675,...,1440.2025,54655.684875,54655.684875,0.026350,0.000694,14967.0756,-1.831072e+06,-1.831072e+06,-0.008174,0.000067


10. Create a Y_sample variable containing the target values corresponding to the rows that were kept in X_sample

In [15]:
Y_sample = Y.loc[X_sample.index]
Y_sample.head()

1523     3.068
2704     0.675
2047     1.272
11174    2.052
1763     1.341
Name: Price, dtype: float64

11. Use the following help to translate the longitude and latitude of the data to find the cities corresponding to each observation: [geopy](https://pypi.org/project/geopy)

In [16]:
!pip install geopy

In [17]:
# Example of how to get the adress from a given pair of latitude/longitude coordinates
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="yet_another_app")
location = geolocator.reverse("52.509669, 13.376294")
loc_dict = dict(location.raw)
loc_dict["address"]

{'tourism': 'Potsdamer Platz',
 'road': 'Fontaneplatz',
 'suburb': 'Tiergarten',
 'borough': 'Mitte',
 'city': 'Berlin',
 'ISO3166-2-lvl4': 'DE-BE',
 'postcode': '10785',
 'country': 'Deutschland',
 'country_code': 'de'}

In [18]:
# Use geopy to extract the city of each row in the sample dataset
X_sample["City"] = 0
for i, row in X_sample.iterrows():
    geolocator = Nominatim(user_agent="yet_another_app_2")
    location = geolocator.reverse("{}, {}".format(X_sample.loc[i, "Latitude"], X_sample.loc[i, "Longitude"]), 
                                  timeout = None)
    loc_dict = dict(location.raw)
    try:
        X_sample.loc[i, "City"] = loc_dict["address"]["city"]
    except:
        try:
            X_sample.loc[i, "City"] = loc_dict["address"]["town"]
        except:
            try:
                X_sample.loc[i, "City"] = loc_dict["address"]["village"]
            except:
                pass
# If city was not found, replace by "Unknown"
X_sample.loc[X_sample['City'] == 0, 'City'] = "Unknown"

In [19]:
X_sample.describe(include='all')

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedInc_2,MedInc_3,...,Latitude_3,Latitude_4,Latitude_inverse,Latitude_inverse2,Longitude_2,Longitude_3,Longitude_4,Longitude_inverse,Longitude_inverse2,City
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.00000,150.000000,150.000000,150.000000,...,150.000000,150.000000,150.000000,150.000000,150.000000,1.500000e+02,1.500000e+02,150.000000,150.000000,150
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
mean,3.788229,28.793333,5.274602,1.074467,1453.853333,2.962908,35.43540,-119.386600,16.855012,85.385706,...,44965.375344,44965.375344,0.028316,0.000804,14256.981259,-1.703008e+06,-1.703008e+06,-0.008378,0.000070,NaN
std,1.587811,11.718337,1.169843,0.151015,1216.489803,0.678291,2.09814,1.961286,13.992479,106.107927,...,8206.187950,8206.187950,0.001624,0.000091,469.795215,8.441531e+04,8.441531e+04,0.000137,0.000002,NaN
min,0.768500,4.000000,1.875000,0.843284,88.000000,1.567164,32.61000,-124.160000,0.590592,0.453870,...,34677.868581,34677.868581,0.024528,0.000602,13317.160000,-1.914014e+06,-1.914014e+06,-0.008666,0.000065,NaN
25%,2.663500,19.250000,4.503833,1.006466,772.250000,2.491288,33.94000,-121.475000,7.094256,18.895676,...,39096.286984,39096.286984,0.026564,0.000706,13905.716025,-1.792507e+06,-1.792507e+06,-0.008480,0.000068,NaN
50%,3.527500,29.000000,5.176892,1.055428,1181.500000,2.873880,34.18500,-118.385000,12.443257,43.893592,...,39949.079845,39949.079845,0.029253,0.000856,14015.008450,-1.659167e+06,-1.659167e+06,-0.008447,0.000071,NaN
75%,4.806725,36.000000,6.110404,1.095928,1672.000000,3.305331,37.64500,-117.922500,23.112218,111.166496,...,53348.538322,53348.538322,0.029464,0.000868,14756.176300,-1.639797e+06,-1.639797e+06,-0.008232,0.000072,NaN


12. Make a train/test splitting from X_sample and Y_sample

In [20]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X_sample, Y_sample, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



13. What preprocessings are necessary now ? The cells below implement the preprocessings, read it carefully and check what is done

In [21]:
categorical_features = ['City']
numeric_features = [c for c in X_sample.columns if c != 'City']

In [22]:
# Create transformer for numeric features
numeric_transformer = StandardScaler()

In [23]:
# Create transformer for categorical features
categorical_transformer = OneHotEncoder(drop='first', handle_unknown = 'ignore') # ignore if unknown categories are found in test set

In [24]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [25]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
X_train = preprocessor.fit_transform(X_train)
print('...Done.')

# Preprocessings on test set
print("Performing preprocessings on test set...")
X_test = preprocessor.transform(X_test) # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')

Performing preprocessings on train set...
...Done.
Performing preprocessings on test set...
...Done.


/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


14. Train a regression model and evaluate the performances. Are you satisfied?

In [26]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

# Print R^2 scores
print("R2 score on training set : ", regressor.score(X_train, Y_train))
print("R2 score on test set : ", regressor.score(X_test, Y_test))

Train model...
...Done.
R2 score on training set :  0.9999999999999997
R2 score on test set :  -828446.9266603033


**The R2 score is really good on the train set, but very disappointing on the test set. Actually, this problem is called "overfitting" and it's the subject of tomorrow's lecture! Spoiler: it's related to the balance between the number of features included in the model and the number of rows used for training. As we're working with a little sample, the model has not seen enough examples while training, and then it's not able to generalize well on the test set 🤓**